In [76]:
import sqlite3
import pandas as pd
from Backtesting import Backtest as bt, data_retriever_util as dr
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from Backtesting import utils as btutil

# Connect to the source SQLite database
def read_data():
    source_db_path = "C:\\Users\\Dhanika Dewan\\Documents\\GitHub\\StockBuddyGenAI\\src\\Data\\NSE_Yahoo_9_FEB_24.sqlite"
    source_conn = sqlite3.connect(source_db_path)

    # Read the data into a pandas DataFrame
    query = "SELECT * FROM NSE WHERE ticker = 'TATAMOTORS.NS'"
    df = pd.read_sql(query, source_conn, parse_dates=['Date'])

    # Close the connection to the source database
    source_conn.close()
    
    return df

def ema_column(data,i):
    data[f'ema_{i}'] = data['close'].ewm(span=i, adjust=False).mean()

# Function to calculate MACD, Signal Line, and MACD Histogram
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    ema_column(data, short_window)
    ema_column(data, long_window)
    data['macd_12_26'] = data['ema_12'] - data['ema_26']
    data['signal_line_12_26'] = data['macd_12_26'].ewm(span=signal_window, adjust=False).mean()
    data['macd_histogram_12_26'] = data['macd_12_26'] - data['signal_line_12_26']
    return data

# Function to implement MACD strategy with Stop-Loss and print statements
def implement_macd_strategy(data, stop_loss_percentage=0.0):
    data = calculate_macd(data)  # Calculate MACD within this function
    buy_signals = [float('nan')] * len(data)  # Initialize with NaNs of DataFrame length
    sell_signals = [float('nan')] * len(data)  # Initialize with NaNs of DataFrame length
    triggers = ['H'] * len(data)  # Initialize with 'H' of DataFrame length
    position = None  # None means no position, 1 means holding stock, 0 means not holding stock
    buy_price = 0  # Track the price at which the stock was bought

    for i in range(1, len(data)):
        flag = False
        # Entry Condition
        if (data['macd_12_26'].iloc[i] > data['signal_line_12_26'].iloc[i] and 
            data['macd_histogram_12_26'].iloc[i] > 0 and
            data['macd_12_26'].iloc[i] > 0):
            flag = True
            if position != 1:
                buy_signals[i] = data['close'].iloc[i]
                sell_signals[i] = float('nan')
                triggers[i] = 'B'
                position = 1
                buy_price = data['close'].iloc[i]

        # Exit Condition based on MACD
        if (data['macd_12_26'].iloc[i] < data['signal_line_12_26'].iloc[i] or
              data['macd_histogram_12_26'].iloc[i] < 0 or
              data['macd_12_26'].iloc[i] < 0):
            flag = True
            if position == 1:
                buy_signals[i] = float('nan')
                sell_signals[i] = data['close'].iloc[i]
                triggers[i] = 'S'
                position = 0

        # Exit Condition based on Stop-Loss
        if data['close'].iloc[i] < buy_price * (1 - stop_loss_percentage):
            flag = True
            if position == 1:
                buy_signals[i] = float('nan')
                sell_signals[i] = data['close'].iloc[i]
                triggers[i] = 'S'
                position = 0

        if not flag:
            buy_signals[i] = float('nan')
            sell_signals[i] = float('nan')
            triggers[i] = 'H'

    # Assign lists to DataFrame columns
    data['buy_signal'] = buy_signals
    data['sell_signal'] = sell_signals
    data['Trigger'] = triggers

    return data


df = read_data()
df = implement_macd_strategy(df)

# Perform a simple backtest
result = bt.simpleBacktest(df)
print(result)

# Plot function
def plotGraph(df, stockName="No name"):
    stockName = stockName[:-3]  # Assuming the stockname will be Ticker data.(stockName.NS)

    # Initialize the figure with subplots
    fig = make_subplots(
        rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02,
        subplot_titles=(stockName + " Historical Data", 'Volume', 'MACD'),
        row_heights=[0.6, 0.2, 0.2], specs=[[{"secondary_y": False}], [{"secondary_y": False}], [{"secondary_y": False}]]
    )

    # Define colors for increasing and decreasing candles
    increasing_color = 'green'
    decreasing_color = 'red'

    # Add the candlestick chart to the first subplot
    fig.add_trace(go.Candlestick(
        x=df['Date'], open=df['Open'], high=df['high'], low=df['low'], close=df['close'],
        increasing=dict(line=dict(color=increasing_color, width=1), fillcolor=increasing_color),
        decreasing=dict(line=dict(color=decreasing_color, width=1), fillcolor=decreasing_color),
        name="Candlestick"
    ), row=1, col=1)

    # Add volume trace to the second subplot
    colors = ['green' if close >= open_ else 'red' for open_, close in zip(df['Open'], df['close'])]
    fig.add_trace(go.Bar(x=df['Date'], y=df['Volume'], marker_color=colors, name='Volume'), row=2, col=1)

    # Add MACD line to the third subplot
    fig.add_trace(go.Scatter(x=df['Date'], y=df['macd_12_26'], mode='lines', name='MACD'), row=3, col=1)
    # Add signal line to the third subplot
    fig.add_trace(go.Scatter(x=df['Date'], y=df['signal_line_12_26'], mode='lines', name='Signal Line'), row=3, col=1)
    # Add MACD histogram to the third subplot
    fig.add_trace(go.Bar(x=df['Date'], y=df['macd_histogram_12_26'], name='MACD Histogram'), row=3, col=1)

    # Customize layout
    fig.update_layout(
        height=800, title=stockName,
        xaxis_title='Date', yaxis_title='Price',
        xaxis_rangeslider_visible=False, showlegend=True
    )

    return fig

# Call the plot function
fig = plotGraph(df, stockName='TATAMOTORS.NS')

# Add buy/sell signals to the graph
fig = btutil.addBuySell2Graph(df, fig)

# Convert the figure to JSON
plotly_json = pio.to_json(fig, pretty=True)
result["plotlyJson"] = plotly_json

# Display the figure
fig.show()


False
8442
{'Win Rate [%]': 35.809312638580934, 'Net Profit/Loss [$]': 73936.36864318838, 'Total Trades': 902, 'Winning Trades': 323.00000000000006}
